# Experiment E21 E22

#### Dependencies

In [71]:
import csv
import re
import os
import json
from openai import OpenAI
from tenacity import (retry, stop_after_attempt, wait_random_exponential)

### Data Processing

In [72]:
# List that hold the records
data = []

# Opening the CSV file
with open('Dataset/ferretti_instrument.csv', mode ='r')as file:
   
  # reading the CSV file
  csvFile = csv.reader(file)
  
  # displaying the contents of the CSV file
  for lines in csvFile:
      data.append(lines)

In [73]:
#for saving the results 
fields_E21 = ['predicate','role','sentence_1','fit_score_1','sentence_2','fit_score_2','sentence_3','fit_score_3','sentence_4','fit_score_4','sentence_5','fit_score_5','actual_fit','exp_fit']
filename_E21 = 'Result_final/E21_ferInst_llama13b_results.csv'
    
fields_E22 = ['predicate','role','sentence_1','fit_score_1','sentence_2','fit_score_2','sentence_3','fit_score_3','sentence_4','fit_score_4','sentence_5','fit_score_5','actual_fit','exp_fit']
filename_E22 = 'Result_final/E22_ferInst_llama13b_results.csv'

In [74]:
if not os.path.exists(filename_E21):
    with open(filename_E21, 'a') as csvfile: 
        # creating a csv writer object  
        csvwriter = csv.writer(csvfile) 

        # writing the fields  
        csvwriter.writerow(fields_E21)

if not os.path.exists(filename_E22):
    with open(filename_E22, 'a') as csvfile: 
        # creating a csv writer object  
        csvwriter = csv.writer(csvfile) 

        # writing the fields  
        csvwriter.writerow(fields_E22)

### Models

In [75]:
openai_api_key = "EMPTY"
openai_api_base = "http://0.0.0.0:8000/v1"

client = OpenAI(
    # defaults to os.environ.get("OPENAI_API_KEY")
    api_key=openai_api_key,
    base_url=openai_api_base,
)

In [76]:
model_engine= "codellama/CodeLlama-13b-Instruct-hf"
temp = 0.0
top_p = 0.95

In [77]:
@retry(wait=wait_random_exponential(min=1, max=40), stop=stop_after_attempt(3))
def chat_with_functions(conversation: [{}]):
  completion = client.chat.completions.create(
      model=model_engine,
      messages=conversation,
      response_format={ "type": "json_object" },
      temperature= temp,
      top_p = top_p,
    )
 
  return completion.choices[0].message.content

### Help Function

In [78]:
def generate_sentence(conversation: [{}], predicate: str, argument: str, roleType: str):
    prompt_generate_sentence = f"Generate five sentences that semantically fit the given predicate '{predicate}', argument '{argument}', and the role of {roleType}. Reply only with a JSON list containing the key 'sentences' with the five sentences each is semantically coherent with the given predicate, argument, and role. Do not reply with anything else other than the JSON list."
    conversation.append({"role": "user", "content": prompt_generate_sentence})
    response_generate_sentence = chat_with_functions(conversation = conversation)

    '''TRACING'''
    print('response_generate_sentence', response_generate_sentence)
    
    sentences = json.loads(response_generate_sentence)["sentences"]
    conversation.append({"role": "assistant", "content": ",".join(sentences)})

    return sentences

In [79]:
def semantic_coherent (conversation: [{}], sentence: str):
    prompt_check_semantic = f"Is the given sentence '{sentence} semantically coherent? Reply only with JSON list has the key 'Is Fit' containing 'Yes' or 'No'. Do not reply with anything else other than the JSON list."
    conversation.append({"role": "user", "content": prompt_check_semantic})
    response_check_semantic = chat_with_functions(conversation = conversation) 
    
    '''TRACING'''
    print('response_check_semantic', response_check_semantic)
    
    is_json_format = is_json(response_check_semantic)
    
    if is_json_format:
        check_semantic = json.loads(response_check_semantic)["Is Fit"]      
    else:
        check_semantic = response_check_semantic

    conversation.append({"role": "assistant", "content": f"{check_semantic}"})

    if 'No' in response_check_semantic:
        return False
    else:
        return True

In [88]:
def textual_fit_scoring_with_senetence(conversation: [{}], sentence: str, predicate: str, argument: str, roleType: str):
    prompt_fit_score_textual= f"Given the following sentence '{sentence}', for the predicate '{predicate}', how much does the argument '{argument}' fit the role of {roleType}?  Reply only with JSON list has the key 'Fit score' containing a value that is one of 'Near-Perfect', 'High', 'Medium', 'Low' or 'Near-Impossible'. Do not reply with anything else other than the JSON list."
    conversation.append({"role": "user", "content": prompt_fit_score_textual})
    response_fit_score_textual =  chat_with_functions(conversation = conversation)                

    '''TRACING'''
    print('response_fit_score_textual',response_fit_score_textual)
    
    is_json_format = is_json(response_fit_score_textual)
    
    if is_json_format:
        fit_score_textual = json.loads(response_fit_score_textual)['Fit score']    
    else:
        fit_score_textual = response_fit_score_textual
    
    conversation.append({"role": "assistant", "content": f"{fit_score_textual}"})
    fit_score = textual_to_numerical_scale(fit_score_textual)

    return fit_score

In [110]:
def numerical_fit_scoring_with_senetence(conversation: [{}], sentence: str, predicate: str, argument: str, roleType: str):
    prompt_fit_score= f"Given the following sentence '{sentence}', for the predicate '{predicate}', how much does the argument '{argument}' fit the role of {roleType}?  Reply only with JSON list has the key 'Fit score' containing a value that is a float number from 0 to 1. Do not reply with anything else other than the JSON list. "
    conversation.append({"role": "user", "content": prompt_fit_score})
    response_fit_score =  chat_with_functions(conversation = conversation)                

    '''TRACING'''
    print('response_fit_score', response_fit_score)
    
    is_json_format = is_json(response_fit_score)
    
    print('jjjj', is_json_format)
        
    if is_json_format:
        fit_score = json.loads(response_fit_score)['Fit score']    
    else:
        fit_score = response_fit_score
    
    conversation.append({"role": "assistant", "content": f"{fit_score}"})

    return fit_score

In [95]:
def textual_fit_scoring_with_backoff(conversation: [{}], predicate: str, argument: str, roleType: str):
    prompt_fit_score_textual= f"Given the predicate '{predicate}', how much does the argument '{argument}' fit the role of {roleType}?  Reply only with JSON list has the key 'Fit score' containing a value that is one of 'Near-Perfect', 'High', 'Medium', 'Low' or 'Near-Impossible'. Do not reply with anything else other than the JSON list."
    conversation.append({"role": "user", "content": prompt_fit_score_textual})
    response_fit_score_textual =  chat_with_functions(conversation = conversation)                
    
    '''TRACING'''
    print('response_fit_score_textual', response_fit_score_textual)
    
    is_json_format = is_json(response_fit_score_textual)
    
    if is_json_format:
        fit_score_textual = json.loads(response_fit_score_textual)['Fit score']
    else:
        fit_score_textual = response_fit_score_textual
        
    ## CHECK
    conversation.append({"role": "assistant", "content": f"{fit_score_textual}"})
    
    fit_score = textual_to_numerical_scale(fit_score_textual)
    fit_score /= 2

    return fit_score

In [109]:
def numerical_fit_scoring_with_backoff(conversation: [{}], predicate: str, argument: str, roleType: str):
    prompt_fit_score= f"Given the predicate '{predicate}', how much does the argument '{argument}' fit the role of {roleType}? Reply only with JSON list has the key 'Fit score' containing a value that is a float number from 0 to 1. Do not reply with anything else other than the JSON list."
    conversation.append({"role": "user", "content": prompt_fit_score})
    response_fit_score = chat_with_functions(conversation = conversation)                 
    
    '''TRACING'''
    print('response_fit_score', response_fit_score)
    
    is_json_format = is_json(response_fit_score)
    
    if is_json_format:
        fit_score = float(json.loads(response_fit_score)['Fit score'])
    else:
        fit_score = float(response_fit_score_textual)
        
    conversation.append({"role": "assistant", "content": f"{fit_score}"})
    fit_score /= 2

    return fit_score

In [103]:
def is_json(myjson):
    try: json.loads(myjson)
    except ValueError as e: 
        return False
    
    return True

In [98]:
def rowExist(result_file_name, predicate, argument):
    with open(result_file_name, mode ='r')as file:
        csvFile = csv.reader(file)
        for line in csvFile:
            if predicate in line and argument in line:
                return True
    return False

In [99]:
def textual_to_numerical_scale(level):
    if "Near-Impossible" in level:
        return 0.0
    elif "Low" in level:
        return 0.25
    elif "Medium" in level:
        return 0.5
    elif "High" in level:
        return 0.75
    elif "Near-Perfect" in level:
        return 1.0

### Experiment

In [111]:
for i in range(5):  # This must be for all datat 'len(data)' but for now just 5 records 
    
    '''Data Extraction'''
    predicate = data[i][0]
    argument = data[i][1]
    roleType = 'Instrument'
    actual_fit = data[i][2]

    '''Check if this record is exists already'''
    IsExist_E21 = False
    IsExist_E22 = False 
    
    if os.path.exists(filename_E21) and  os.path.exists(filename_E22):
        IsExist_E21 = rowExist(filename_E21, predicate,argument)
        IsExist_E22 = rowExist(filename_E22, predicate,argument)

        if IsExist_E21 and IsExist_E22:
            continue
    
    '''TRACING'''
    print('New Record')
    print('--------------------------------------')
    print(predicate,argument)
    
    '''Result Preparation'''    
    if not IsExist_E21:
        result_E21 = [predicate,argument]
    if not IsExist_E22:
        result_E22 = [predicate,argument] 

    '''Chat Completion Needed Parameter'''
    system_prompt = "You are a linguist who can understand the semantic roles and can provide a rating on the semantic fit of predicate-arguments for a specific semantic role, given the predicate, the argument and the semantic role."
    conversation = [{"role": "system", "content": system_prompt}] 

    '''Sentence Generation Part'''
    sentence_no = 5
    sentences = generate_sentence(conversation= conversation, predicate= predicate, argument= argument, roleType=roleType)

    '''TRACING'''
    print(sentences)
    
    '''Prompting'''
    avg_E21 = 0
    avg_E22 = 0

    for j in sentences:

        '''Checking Semantic Coherent'''
        is_semantic = semantic_coherent(conversation= conversation, sentence= j)      
            
        '''Not Sematically Fit Sentence'''
        if not is_semantic:

            '''Back Off'''
            
            '''TRACING'''
            print(j, '\t Not Sematically Fit Sentence')
            
            # as the sentnece is not semantically fit, do not consider and add it as an empty in the result
            if not IsExist_E21:
                result_E21.append('')
            if not IsExist_E22:
                result_E22.append('')
                    
            if not IsExist_E22:
                fit_score = textual_fit_scoring_with_backoff(conversation= conversation, predicate= predicate, argument= argument, roleType=roleType)

                '''TRACING'''
                print(fit_score)
                
                result_E22.append(fit_score)
                avg_E22 += fit_score

            if not IsExist_E21:
                fit_score = numerical_fit_scoring_with_backoff(conversation= conversation, predicate= predicate, argument= argument, roleType=roleType)

                '''TRACING'''
                print(fit_score)
                
                result_E21.append(fit_score)
                avg_E21 += fit_score

        '''Sematically Fit Sentence'''
        if is_semantic:    

            
            # consider the sentence
            if not IsExist_E21:
                result_E21.append(j)
            if not IsExist_E22:
                result_E22.append(j)

            if not IsExist_E22:
                fit_score = textual_fit_scoring_with_senetence(conversation= conversation, sentence= j, predicate= predicate, argument= argument, roleType=roleType)

                '''TRACING'''
                print(fit_score)
                
                result_E22.append(fit_score)
                avg_E22 += fit_score

            if not IsExist_E21:
                fit_score = numerical_fit_scoring_with_senetence(conversation= conversation, sentence= j, predicate= predicate, argument= argument, roleType=roleType)

                '''TRACING'''
                print(fit_score)
                
                result_E21.append(fit_score)
                avg_E21 += fit_score

    '''Add the results'''
    result_E21.append(actual_fit)
    result_E21.append(round(avg_E21/sentence_no,2))

    result_E22.append(actual_fit)
    result_E22.append(round(avg_E22/sentence_no,2))

    '''TRACING'''
    print(predicate, argument, roleType, 'fit_score_final', round(avg_E21/sentence_no,2) ,  round(avg_E22/sentence_no,2))

    '''ADD IT TO THE RESULT DATASET'''
    if not IsExist_E21:
        with open(filename_E21, 'a') as csvfile: 
            # creating a csv writer object  
            csvwriter = csv.writer(csvfile) 

            # writing the data rows  
            csvwriter.writerow(result_E21) 
        
    if not IsExist_E22:
        with open(filename_E22, 'a') as csvfile: 
            # creating a csv writer object  
            csvwriter = csv.writer(csvfile) 

            # writing the data rows  
            csvwriter.writerow(result_E22) 

New Record
--------------------------------------
serve plate
response_generate_sentence   {
"sentences": [
"The waiter served the plate of spaghetti to the customer.",
"The chef served the plate of sushi to the customer.",
"The server served the plate of fries to the customer.",
"The waiter served the plate of soup to the customer.",
"The chef served the plate of salad to the customer."
]
}
['The waiter served the plate of spaghetti to the customer.', 'The chef served the plate of sushi to the customer.', 'The server served the plate of fries to the customer.', 'The waiter served the plate of soup to the customer.', 'The chef served the plate of salad to the customer.']
response_check_semantic   {
"Is Fit": "Yes"
}
response_fit_score_textual  {
"Fit score": "High"
}
0.75
response_fit_score  {
"Fit score": 0.8
}
jjjj True
0.8
response_check_semantic   Yes
response_fit_score_textual   High
0.75
response_fit_score  0.9
jjjj True


TypeError: 'float' object is not subscriptable